# Advanced Modeling for BTC Daily Features

Plan :
1. Implémenter des versions avancées des algos vus en cours (ElasticNet, GradientBoosting/HistGB, RandomForest/ExtraTrees).
2. Définir le plan apprentissage/test : choix des données, split chrono, contrôle de l'overfitting.
3. Analyser et critiquer les résultats avec MAE/RMSE/MAPE.
4. Combiner plusieurs algos (stacking/average) pour la décision.
5. Choisir un algo hors cours (réseau de neurones MLP) avec référence.
6. Expliquer l'algo et justifier le choix (référence scientifique).
7. Implémenter, évaluer et comparer avec les résultats précédents et le baseline.


## 0. Imports & Helpers
- Chargement des librairies
- Fonctions utilitaires (RMSE/MAPE, split chrono, évaluation)


In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import ElasticNet, Ridge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, StackingRegressor
from sklearn.neural_network import MLPRegressor

pd.set_option('display.width', 200)

def rmse(y_true, y_pred):
    return float(mean_squared_error(y_true, y_pred) ** 0.5)

def mape(y_true, y_pred):
    pct = np.abs((y_true - y_pred) / y_true)
    pct = pd.Series(pct).replace([np.inf, -np.inf], np.nan).dropna()
    return float(pct.mean() * 100)

def chrono_split(X, y, train_ratio=0.7, val_ratio=0.15):
    n = len(X)
    train_end = int(n * train_ratio)
    val_end = int(n * (train_ratio + val_ratio))
    return (X.iloc[:train_end], y.iloc[:train_end],
            X.iloc[train_end:val_end], y.iloc[train_end:val_end],
            X.iloc[val_end:], y.iloc[val_end:])

def evaluate_model(name, model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return {
        'model': name,
        'MAE': mean_absolute_error(y_test, preds),
        'RMSE': rmse(y_test, preds),
        'MAPE%': mape(y_test, preds),
    }


### Explication (Imports & helpers)
- Fonctions utilitaires partagées : RMSE(Root Mean Squared Error)/MAPE (Mean Absolute Percentage Error)  pour les métriques, split chronologique pour éviter la fuite temporelle, `evaluate_model` pour entraîner + scorer.
- Les pipelines scaler+modèle assurent une standardisation cohérente quand nécessaire (linéaire, MLP).


## 1. Données et cible
- Chargement `btc_features_daily.csv`
- Tri chronologique
- Cible : close du lendemain (`target_close_next`)


In [2]:

df = pd.read_csv(Path('btc_features_daily.csv'), parse_dates=['timestamp']).sort_values('timestamp')
df['target_close_next'] = df['close'].shift(-1)
df = df.iloc[:-1]

corr = (
    df.drop(columns=['timestamp'])
      .corr()['target_close_next']
      .drop('target_close_next')
      .abs()
      .sort_values(ascending=False)
)
print('Top 10 corrélations (absolues):')
print(corr.head(10))







Top 10 corrélations (absolues):
adj close       0.999377
close           0.999377
typ_price       0.999328
ohlc4           0.999259
median_price    0.999248
high            0.999135
low             0.999077
open            0.998810
wma_10          0.998472
ma_7            0.998334
Name: target_close_next, dtype: float64


In [3]:
data_path = Path('btc_features_daily.csv')
df = pd.read_csv(data_path, parse_dates=['timestamp']).sort_values('timestamp').reset_index(drop=True)
print(f"Loaded {len(df):,} rows, {len(df.columns)} columns")
print('Missing ratio (top 5):')
print(df.isna().mean().head())

# Cible = close J+1
df['target_close_next'] = df['close'].shift(-1)
df = df.iloc[:-1]
feature_cols = [c for c in df.columns if c not in ['timestamp', 'target_close_next']]
X = df[feature_cols]
y = df['target_close_next']

X_train, y_train, X_val, y_val, X_test, y_test = chrono_split(X, y)
print(f"Splits -> train {len(X_train)}, val {len(X_val)}, test {len(X_test)}")


Loaded 3,973 rows, 52 columns
Missing ratio (top 5):
timestamp    0.0
adj close    0.0
close        0.0
high         0.0
low          0.0
dtype: float64
Splits -> train 2780, val 596, test 596


## 1bis. Ajout des données de sentiment
On ajoute le Fear & Greed Index hebdomadaire comme feature supplémentaire :
- chargement de `sentiment_dataset.csv` si présent, sinon `sentiment_fng_dataset.csv`
- resample en quotidien avec forward-fill
- fusion avec les features BTC, puis recalcul de la cible et des splits


In [4]:
# Fusion des données de sentiment (hebdo) en daily
from pathlib import Path

sent_paths = [Path('sentiment_dataset.csv'), Path('sentiment_fng_dataset.csv')]
sent_path = None
for p in sent_paths:
    if p.exists():
        sent_path = p
        break
if sent_path is None:
    raise FileNotFoundError('Aucun fichier de sentiment trouvé (sentiment_dataset.csv ou sentiment_fng_dataset.csv)')

sent = pd.read_csv(sent_path)
if 'date' not in sent.columns:
    sent = sent.rename(columns={sent.columns[0]: 'date'})
sent['date'] = pd.to_datetime(sent['date'])
# Nettoyage des noms de colonnes
sent.columns = [c.strip().lower() for c in sent.columns]
# Passage en daily avec ffill
sent_daily = sent.set_index('date').sort_index().resample('D').ffill().reset_index()

# Fusion sur la date normalisée
df['date'] = df['timestamp'].dt.normalize()
df = df.merge(sent_daily, on='date', how='left')
sent_cols = [c for c in sent_daily.columns if c != 'date']
for c in sent_cols:
    df[c] = df[c].ffill().bfill()

df = df.drop(columns=['date'])

# Recalcule cible + splits avec les features enrichies
df['target_close_next'] = df['close'].shift(-1)
df = df.iloc[:-1]
feature_cols = [c for c in df.columns if c not in ['timestamp', 'target_close_next']]
X = df[feature_cols]
y = df['target_close_next']
X_train, y_train, X_val, y_val, X_test, y_test = chrono_split(X, y)
print(f"Fusion sentiment: features = {len(feature_cols)}, sentiment cols = {sent_cols}")
print(f"Splits -> train {len(X_train)}, val {len(X_val)}, test {len(X_test)}")


Fusion sentiment: features = 52, sentiment cols = ['fear_and_greed_index']
Splits -> train 2779, val 596, test 596


### Explication (Fusion sentiment)
- On aligne le Fear & Greed Index (hebdo) sur une fréquence quotidienne via `resample('D').ffill()`.
- Les valeurs manquantes hors plage sont comblées par `ffill()/bfill()` pour ne pas perdre d'observations.
- Les nouvelles colonnes de sentiment sont ajoutées aux features avant de recalculer la cible et les splits.


### Explication (Données & cible)
- On charge `btc_features_daily.csv`, on trie par date, pas de valeurs manquantes sur les principales colonnes.
- Cible = close du lendemain (`target_close_next`) : formulation régressive, sensible à l'autocorrélation.
- Split 70/15/15 chronologique pour reproduire un scénario temps-réel.


## 2. Corrélations et contrôle de l'overfitting
- Corrélations très élevées entre niveaux de prix (indique redondance)
- Plan pour limiter l'overfit :
  - Standardisation pour modèles linéaires/MLP
  - Hyperparamètres contraints (profondeur, min_samples_leaf)
  - Split chrono (pas de fuite temporelle)


In [5]:
corr = (
    df.drop(columns=['timestamp'])
      .corr()['target_close_next']
      .drop('target_close_next')
      .abs()
      .sort_values(ascending=False)
)
print('Top 8 corrélations absolues avec la cible:')
print(corr.head(8))


Top 8 corrélations absolues avec la cible:
adj close       0.999376
close           0.999376
typ_price       0.999327
ohlc4           0.999258
median_price    0.999247
high            0.999133
low             0.999075
open            0.998809
Name: target_close_next, dtype: float64


### Explication (Corrélations & overfitting)
- Les corrélations >0.99 entre prix (close/adj/typ_price) et la cible indiquent une forte redondance.
- Risque : les modèles complexes apprennent surtout à recopier le prix actuel → difficile de généraliser.
- Contre-mesures : régularisation, profondeur limitée, validation temporelle stricte.


## 3. Baseline
Baseline persistance : prédire que le close de demain = close d’aujourd’hui.


In [6]:
baseline_preds = X_test['close']
baseline_res = {
    'model': 'persistence_close',
    'MAE': mean_absolute_error(y_test, baseline_preds),
    'RMSE': rmse(y_test, baseline_preds),
    'MAPE%': mape(y_test, baseline_preds),
}
baseline_res


{'model': 'persistence_close',
 'MAE': 1475.7585596686242,
 'RMSE': 2027.3115008294158,
 'MAPE%': 1.7420895365842408}

### Explication (Baseline)
- Cette baseline prédit simplement que le close de demain = close d'aujourd'hui.
- Si ses MAE/RMSE/MAPE sont très bas, cela signifie que la série est fortement autocorrélée et difficile à battre.
- Tout modèle avancé doit faire mieux (MAE/RMSE plus petits, MAPE plus faible) pour être intéressant.


## 4. Modèles avancés (versions cours)
- ElasticNet (L1+L2) pour gérer colinéarité et sélection douce.
- Ridge (référence linéaire régularisée).
- GradientBoostingRegressor et HistGradientBoostingRegressor (plus robuste, bins).
- RandomForestRegressor et ExtraTreesRegressor (arbres bagging/variance réduite).


In [7]:
models = [
    ('elasticnet', make_pipeline(StandardScaler(), ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=5000))),
    ('ridge', make_pipeline(StandardScaler(), Ridge(alpha=5.0))),
    ('gboost', GradientBoostingRegressor(random_state=0, n_estimators=400, learning_rate=0.02, max_depth=3)),
    ('hist_gb', HistGradientBoostingRegressor(random_state=0, max_depth=4, learning_rate=0.05, max_iter=500)),
    ('rf', RandomForestRegressor(n_estimators=400, random_state=0, n_jobs=-1, max_depth=6, min_samples_leaf=3)),
    ('extra_trees', ExtraTreesRegressor(n_estimators=400, random_state=0, n_jobs=-1, max_depth=6, min_samples_leaf=3)),
]

results = [baseline_res]
for name, model in models:
    results.append(evaluate_model(name, model, X_train, y_train, X_test, y_test))

pd.DataFrame(results).sort_values('RMSE')


,model,MAE,RMSE,MAPE%
0,persistence_close,1475.758560,2027.311501,1.742090
2,ridge,1766.136507,2315.695459,2.037025
1,elasticnet,2338.694596,2922.421378,2.631561
6,extra_trees,25049.640083,31826.364387,24.531465
5,rf,25010.779401,31835.098496,24.474331
3,gboost,26481.603420,32964.672377,26.238465
4,hist_gb,26395.776179,33114.697807,26.031712


### Explication (Baseline)
- Cette baseline prédit simplement que le close de demain = close d'aujourd'hui.
- Si ses MAE/RMSE/MAPE sont très bas, cela signifie que la série est fortement autocorrélée et difficile à battre.
- Tout modèle avancé doit faire mieux (MAE/RMSE plus petits, MAPE plus faible) pour être intéressant.


### Explication (Modèles avancés)
- **ElasticNet/Ridge** : testent si une régularisation linéaire peut extraire un léger signal malgré la colinéarité.
- **GB/HistGB** : capturent des non-linéarités; l'apprentissage lent (petit learning_rate) limite l'overfit.
- **RF/ExtraTrees** : bagging d'arbres, ici contraints (max_depth/min_samples_leaf) pour réduire l'overfit.
- Interprétation : comparez leurs métriques au baseline. Si elles sont plus grandes, ils n'apportent pas de gain; s'ils sont plus petites, ils surpassent la persistance.


## 5. Ensemble learning
On combine plusieurs modèles pour lisser leurs erreurs : moyenne simple et StackingRegressor (ridge meta-learner).


In [8]:
# Fit base models
fitted = {}
for name, model in models:
    model.fit(X_train, y_train)
    fitted[name] = model

# Moyenne simple des prédictions des 4 meilleurs modèles (ElasticNet, Ridge, RF, HistGB)
selected = ['elasticnet','ridge','rf','hist_gb']
preds_avg = np.mean([fitted[n].predict(X_test) for n in selected], axis=0)
ensemble_avg = {
    'model': 'avg_elastic_ridge_rf_histgb',
    'MAE': mean_absolute_error(y_test, preds_avg),
    'RMSE': rmse(y_test, preds_avg),
    'MAPE%': mape(y_test, preds_avg),
}

# Stacking
estimators = [(n, fitted[n]) for n in selected]
stack = StackingRegressor(estimators=estimators, final_estimator=Ridge(alpha=1.0), passthrough=False)
stack.fit(X_train, y_train)
preds_stack = stack.predict(X_test)
ensemble_stack = {
    'model': 'stack_elastic_ridge_rf_histgb',
    'MAE': mean_absolute_error(y_test, preds_stack),
    'RMSE': rmse(y_test, preds_stack),
    'MAPE%': mape(y_test, preds_stack),
}

ensemble_results = [ensemble_avg, ensemble_stack]
pd.DataFrame(results + ensemble_results).sort_values('RMSE')


,model,MAE,RMSE,MAPE%
0,persistence_close,1475.758560,2027.311501,1.742090
8,stack_elastic_ridge_rf_histgb,1700.283300,2248.035931,1.969836
2,ridge,1766.136507,2315.695459,2.037025
1,elasticnet,2338.694596,2922.421378,2.631561
7,avg_elastic_ridge_rf_histgb,13662.807629,17064.742660,13.538915
6,extra_trees,25049.640083,31826.364387,24.531465
5,rf,25010.779401,31835.098496,24.474331
3,gboost,26481.603420,32964.672377,26.238465
4,hist_gb,26395.776179,33114.697807,26.031712


### Explication (Ensemble learning)
- **Moyenne simple** : lisse les erreurs de modèles complémentaires; efficace si les erreurs ne sont pas corrélées.
- **Stacking** : méta-modèle (Ridge) apprend à pondérer les prédictions; peut capturer des interactions entre modèles.
- Lecture : si l'ensemble baisse MAE/RMSE/MAPE vs chaque modèle individuel et vs baseline, il apporte de la robustesse.


## 6. Modèle hors cours : MLP Regressor (réseau de neurones feed-forward)
Référence : Zhang, G. P. (2003), "Time series forecasting using a hybrid ARIMA and neural network model", *Neurocomputing*. Les MLP capturent des relations non linéaires; ici on l’emploie sur les features existantes.


In [9]:
mlp = make_pipeline(
    StandardScaler(),
    MLPRegressor(hidden_layer_sizes=(128, 64), activation='relu', solver='adam', max_iter=500, random_state=0, early_stopping=True)
)

mlp_res = evaluate_model('mlp_regressor', mlp, X_train, y_train, X_test, y_test)

pd.DataFrame(results + ensemble_results + [mlp_res]).sort_values('RMSE')


,model,MAE,RMSE,MAPE%
0,persistence_close,1475.758560,2027.311501,1.742090
8,stack_elastic_ridge_rf_histgb,1700.283300,2248.035931,1.969836
2,ridge,1766.136507,2315.695459,2.037025
1,elasticnet,2338.694596,2922.421378,2.631561
9,mlp_regressor,4826.045582,6028.113499,5.170059
7,avg_elastic_ridge_rf_histgb,13662.807629,17064.742660,13.538915
6,extra_trees,25049.640083,31826.364387,24.531465
5,rf,25010.779401,31835.098496,24.474331
3,gboost,26481.603420,32964.672377,26.238465
4,hist_gb,26395.776179,33114.697807,26.031712


### Explication (MLP hors cours)
- MLP Regressor (réseau de neurones feed-forward) peut modéliser des relations non linéaires complexes.
- Early stopping réduit l'overfit; le scaling est crucial.
- Interprétation : comparez ses métriques à la baseline et aux autres. Si l'erreur est plus élevée, les features actuelles ne suffisent pas; si elle baisse, cela suggère un signal non linéaire exploité.


## 7. Analyse et comparaison
- Comparez chaque modèle au baseline persistance.
- Notez si l’ensemble (avg/stack) ou le MLP apportent un gain.
- Inspectez les métriques (MAE/RMSE/MAPE) et la robustesse.


In [10]:
final_table = pd.DataFrame(results + ensemble_results + [mlp_res]).sort_values('RMSE')
final_table.reset_index(drop=True)


,model,MAE,RMSE,MAPE%
0,persistence_close,1475.758560,2027.311501,1.742090
1,stack_elastic_ridge_rf_histgb,1700.283300,2248.035931,1.969836
2,ridge,1766.136507,2315.695459,2.037025
3,elasticnet,2338.694596,2922.421378,2.631561
4,mlp_regressor,4826.045582,6028.113499,5.170059
5,avg_elastic_ridge_rf_histgb,13662.807629,17064.742660,13.538915
6,extra_trees,25049.640083,31826.364387,24.531465
7,rf,25010.779401,31835.098496,24.474331
8,gboost,26481.603420,32964.672377,26.238465
9,hist_gb,26395.776179,33114.697807,26.031712


### Explication (Analyse finale)
- Triez par RMSE/MAE : plus petit = meilleur. La MAPE donne une lecture en %.
- Identifier le premier modèle qui bat clairement la baseline; sinon conclure que les features sont insuffisantes et qu'il faut changer la cible (retours/direction) ou ajouter des variables exogènes.
- Vérifier la stabilité : si un modèle surperforme mais avec très faible marge, valider en rolling split/val croisée temporelle.


## 8. Variante : cible en rendement (log-return) + sentiment
On teste si formuler la cible en rendement log (close_{t+1}/close_t) permet d'exploiter mieux les signaux, en incluant le sentiment F&G ffill en daily.


In [11]:
# Variante retour log + sentiment (rapide)
import numpy as np
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor

btc = pd.read_csv('btc_features_daily.csv', parse_dates=['timestamp']).sort_values('timestamp').reset_index(drop=True)
sent = pd.read_csv('sentiment_fng_dataset.csv')
if 'date' not in sent.columns:
    sent = sent.rename(columns={sent.columns[0]:'date'})
sent['date'] = pd.to_datetime(sent['date'])
sent.columns = [c.lower() for c in sent.columns]
sent_daily = sent.set_index('date').sort_index().resample('D').ffill().reset_index()

btc['date'] = btc['timestamp'].dt.normalize()
btc = btc.merge(sent_daily, on='date', how='left')
btc['fear_and_greed_index'] = btc['fear_and_greed_index'].ffill().bfill()
btc = btc.drop(columns=['date'])

# Cible : log-return du close t->t+1
y = np.log(btc['close'].shift(-1) / btc['close'])
btc = btc.iloc[:-1]
y = y.iloc[:-1]
feature_cols = [c for c in btc.columns if c not in ['timestamp']]
X = btc[feature_cols]

n = len(X); tr=int(n*0.7); va=int(n*0.85)
X_train, X_val, X_test = X.iloc[:tr], X.iloc[tr:va], X.iloc[va:]
y_train, y_val, y_test = y.iloc[:tr], y.iloc[tr:va], y.iloc[va:]

rmse = lambda yt, yp: mean_squared_error(yt, yp)**0.5
results_ret = []
# baseline : retour nul
pred0 = np.zeros_like(y_test)
results_ret.append({'model':'baseline_zero_return','MAE':mean_absolute_error(y_test,pred0),'RMSE':rmse(y_test,pred0)})

models_ret = [
    ('ridge', make_pipeline(StandardScaler(), Ridge(alpha=5.0))),
    ('elastic', make_pipeline(StandardScaler(), ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=5000))),
    ('hist_gb', HistGradientBoostingRegressor(random_state=0, max_depth=4, learning_rate=0.05, max_iter=500)),
    ('rf', RandomForestRegressor(n_estimators=400, random_state=0, n_jobs=-1, max_depth=6, min_samples_leaf=3)),
]
for name, model in models_ret:
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    results_ret.append({'model':name,'MAE':mean_absolute_error(y_test,pred),'RMSE':rmse(y_test,pred)})

pd.DataFrame(results_ret).sort_values('RMSE')


,model,MAE,RMSE
0,baseline_zero_return,0.017443,0.024094
2,elastic,0.017478,0.024101
4,rf,0.018167,0.024724
3,hist_gb,0.031036,0.037191
1,ridge,0.033007,0.039333


### Explication (rendement log)
- Baseline = retour nul (prédire aucune variation) reste la meilleure ou quasi ex æquo, montrant que le bruit domine.
- Les modèles (Ridge/ElasticNet/HistGB/RF) ne dépassent pas le baseline, malgré la reformulation de la cible et l'ajout du sentiment F&G.
- Conclusion intermédiaire : même avec une cible en rendement et le sentiment, on n'extrait pas de signal prédictif net. Il faut d'autres variables ou une cible différente (classification directionnelle) pour espérer mieux.


### Conclusion rapide
- Si aucun modèle ne bat le baseline : les features n’apportent pas de signal supplémentaire; envisager des cibles en rendements/direction et des variables exogènes.
- Si un ensemble ou le MLP réduit la MAPE/MAE, poursuivre l’affinage (tuning hyperparam, rolling validation).
- Toujours valider en split temporel ou rolling pour éviter la fuite.
